In [1]:
import re
from collections import defaultdict

from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals
 
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words
 
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.edmundson import EdmundsonSummarizer

from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [2]:
books = ["data/Rowling, J.K. - HP 1 - Harry Potter and the Sorcerer's Stone.txt",
         "data/Rowling, J.K. - HP 2 - Harry Potter and the Chamber of Secrets.txt",
         "data/Rowling, J.K. - HP 3 - Harry Potter and the Prisoner of Azkaban.txt",
         "data/Rowling, J.K. - HP 4 - Harry Potter and the Goblet of Fire.txt",
         "data/Rowling, J.K. - HP 5 - Harry Potter and the Order of the Phoenix.txt",
         "data/Rowling, J.K. - HP 6 - Harry Potter and the Half-Blood Prince.txt",
         "data/Rowling, J.K. - HP 7 - Harry Potter and the Deathly Hallows.txt"]

In [3]:
pattern = ("(C H A P T E R (?:[A-Z-][ ]){2,}[A-Z]|"
           "E P I L O G U E)\s+" +                   # Group 1 selects the chapter number
           "([A-Z \n',.-]+)\\b(?![A-Z]+(?=\.)\\b)" + # Group 2 selects the chapter title but excludes all caps word beginning first sentence of the chapter
           "(?![a-z']|[A-Z.])" +                     # chapter title ends before lowercase letters or a period
           "(.*?)" +                                 # Group 3 selects the chapter contents
           "(?=C H A P T E R (?:[A-Z][ ]){2,}|"
           "This\s+book\s+was\s+art\s+directed\s+|"
           "E P I L O G U E)")                       # chapter contents ends with a new chapter, epilogue or the end of book
hp = defaultdict(dict)
for book in books:
    title = book[28:-4]
    with open(book, 'r') as f:
        text = (f.read().replace('&rsquo;', "'")
                        .replace('&lsquo;', "'")
                        .replace('&rdquo;', '"')
                        .replace('&ldquo;', '"')
                        .replace('&mdash;', '—'))
    chapters = re.findall(pattern, text, re.DOTALL)
    chap = 0
    for chapter in chapters:
        chap += 1
        chap_title = chapter[1].replace('\n','')
        chap_text = chapter[2][3:]
        phrase = ' HE-WHO-MUST-NOT-BE-NAMED RETURNS'
        if phrase in chap_title:
            chap_title = chap_title.replace(phrase, '')
            chap_text = phrase[1:] + ' I' + chap_text
        chap_text = re.sub('\n*&bull; [0-9]+ &bull; \n*' + chap_title + ' \n*', '', chap_text, flags=re.IGNORECASE)
        chap_text = re.sub('\n*&bull; [0-9]+ &bull;\s*(CHAPTER [A-Z-]+\s*)|(EPILOGUE)\s*', '', chap_text)
        chap_text = re.sub(' \n&bull; [0-9]+ &bull; \n*', '', chap_text)
        chap_text = re.sub('\s*'.join([word for word in chap_title.split()]), '', chap_text)
        hp[title]['Chapter ' + str(chap)] = (chap_title, chap_text)
hp = dict(hp)
hp["Harry Potter and the Deathly Hallows"]['Epilogue'] = hp["Harry Potter and the Deathly Hallows"].pop('Chapter 37')

### The format of the Harry Potter hp dictionary is as follows:
#### The first book is 'hp1', second is 'hp2', etc.
&nbsp;   
{'hp1': {  
         &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 'Chapter 1': (chapter title, chapter text),  
         &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 'Chapter 2': (chapter title, chapter text),  
         &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 'Chapter 3': (chapter title, chapter text),  
         &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ...  
         &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; }  
 'hp2': {  
         &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 'Chapter 1': (chapter title, chapter text),  
         &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ...  
         &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; }  
 &nbsp;...  
}  

In [4]:
for book in hp:
    print(book)

Harry Potter and the Sorcerer's Stone
Harry Potter and the Chamber of Secrets
Harry Potter and the Prisoner of Azkaban
Harry Potter and the Goblet of Fire
Harry Potter and the Order of the Phoenix
Harry Potter and the Half-Blood Prince
Harry Potter and the Deathly Hallows


In [5]:
language = "english"
sentence_count = 5
 
parser = PlaintextParser(hp["Harry Potter and the Sorcerer's Stone"]['Chapter 1'][1], Tokenizer(language))

In [6]:
from sumy.summarizers.lex_rank import LexRankSummarizer 
summarizer = LexRankSummarizer(Stemmer(language))
summarizer.stop_words = get_stop_words(language)
#Summarize the document with 2 sentences
summary = summarizer(parser.document, sentence_count) 
for sentence in summary:
    print(sentence)

"The Potters, that's right, that's what I heard —" "— yes, their son, Harry —" Mr. Dursley stopped dead.
Twelve times he clicked the Put-Outer, until the only lights left on the whole street were two tiny pinpricks in the distance, which were the eyes of the cat watching him.
Dumbledore slipped the Put-Outer back inside his cloak and set off down the street toward number four, where he sat down on the wall next to the cat.
"But I c-c-can't stand it — Lily an' James dead — an' poor little Harry off ter live with Muggles —" "Yes, yes, it's all very sad, but get a grip on yourself, Hagrid, or we'll be found," Professor McGonagall whispered, patting Hagrid gingerly on the arm as Dumbledore stepped over the low garden wall and walked to the front door.
Dumbledore turned and walked back down the street.


In [7]:
from sumy.summarizers.luhn import LuhnSummarizer
summarizer_1 = LuhnSummarizer(Stemmer(language))
summarizer_1.stop_words = get_stop_words(language)
summary_1 = summarizer_1(parser.document, sentence_count)
for sentence in summary_1:
    print(sentence)

It was now reading the sign that said Privet Drive — no, looking at the sign; cats couldn't read maps or signs.
He didn't see the owls swoop- ing past in broad daylight, though people down in the street did; they pointed and gazed open-mouthed as owl after owl sped overhead.
No one knows why, or how, but they're saying that when he couldn't kill Harry Potter, Voldemort's power somehow broke — and that's why he's gone."
"But I c-c-can't stand it — Lily an' James dead — an' poor little Harry off ter live with Muggles —" "Yes, yes, it's all very sad, but get a grip on yourself, Hagrid, or we'll be found," Professor McGonagall whispered, patting Hagrid gingerly on the arm as Dumbledore stepped over the low garden wall and walked to the front door.
G'night, Professor McGonagall — Professor Dumbledore, sir."


In [8]:
from sumy.summarizers.lsa import LsaSummarizer
summarizer_2 = LsaSummarizer(Stemmer(language))
summarizer_2.stop_words = get_stop_words(language)
summary_2 = summarizer_2(parser.document, sentence_count)
for sentence in summary_2:
    print(sentence)

He dashed back across the road, hurried up to his office, snapped at his secretary not to disturb him, seized his telephone, and had almost finished dialing his home number when he changed his mind.
It seemed that Professor McGonagall had reached the point she was most anxious to discuss, the real reason she had been waiting on a cold, hard wall all day, for neither as a cat nor as a woman had she fixed Dumbledore with such a piercing stare as she did now.
He looked simply too big to be allowed, and so wild — long tangles of bushy black hair and beard hid most of his face, he had hands the size of trash can lids, and his feet in their leather boots were like baby dolphins.
For a full minute the three of them stood and looked at the little bundle; Hagrid's shoulders shook, Professor McGonagall blinked furiously, and the twinkling light that usually shone from Dumbledore's eyes seemed to have gone out.
A breeze ruffled the neat hedges of Privet Drive, which lay silent and tidy under the 

In [9]:
from sumy.summarizers.text_rank import TextRankSummarizer
summarizer_3 = TextRankSummarizer(Stemmer(language))
summarizer_3.stop_words = get_stop_words(language)
summary_3 = summarizer_3(parser.document, sentence_count)
for sentence in summary_3:
    print(sentence)

Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much.
They were the last people you'd expect to be involved in anything strange or mysterious, because they just didn't hold with such nonsense.
Mr. Dursley was the director of a firm called Grunnings, which made drills.
He was a big, beefy man with hardly any neck, although he did have a very large mustache.
Mrs. Dursley was thin and blonde and had nearly twice the usual amount of neck, which came in very useful as she spent so much of her time craning over garden fences, spying on the neighbors.


/anaconda3/lib/python3.6/site-packages/sumy/summarizers/text_rank.py:68: RuntimeWarning: invalid value encountered in true_divide
  weights /= weights.sum(axis=1)[:, numpy.newaxis]


In [10]:
from sumy.summarizers.edmundson import EdmundsonSummarizer
summarizer_4 = EdmundsonSummarizer(Stemmer(language))
summarizer_4.stop_words = get_stop_words(language)
summarizer_4.bonus_words = hp["Harry Potter and the Sorcerer's Stone"]['Chapter 1'][0].split()
summarizer_4.stigma_words = ['zdfgthdvndadv']
summarizer_4.null_words = ['zdfgthdvndadv']
summary_4 = summarizer_4(parser.document, sentence_count)
for sentence in summary_4:
    print(sentence)

The Dursleys shuddered to think what the neighbors would say if the Potters arrived in the street.
When Dudley had been put to bed, he went into the living room in time to catch the last report on the evening news: "And finally, bird-watchers everywhere have reported that the nation's owls have been behaving very unusually today.
Twelve times he clicked the Put-Outer, until the only lights left on the whole street were two tiny pinpricks in the distance, which were the eyes of the cat watching him.
Dumbledore slipped the Put-Outer back inside his cloak and set off down the street toward number four, where he sat down on the wall next to the cat.
He couldn't know that at this very moment, people meeting in secret all over the country were holding up their glasses and saying in hushed voices: "To Harry Potter — the boy who lived!"


In [11]:
from sumy.summarizers.edmundson import EdmundsonSummarizer
summarizer_4 = EdmundsonSummarizer(Stemmer(language))
summarizer_4.stop_words = get_stop_words(language)
summarizer_4.bonus_words = hp["Harry Potter and the Sorcerer's Stone"]['Chapter 1'][0].split()
summarizer_4.stigma_words = ['zdfgthdvndadv']
summarizer_4.null_words = stop_words
summary_4 = summarizer_4(parser.document, sentence_count)
for sentence in summary_4:
    print(sentence)

The Dursleys shuddered to think what the neighbors would say if the Potters arrived in the street.
When Dudley had been put to bed, he went into the living room in time to catch the last report on the evening news: "And finally, bird-watchers everywhere have reported that the nation's owls have been behaving very unusually today.
Twelve times he clicked the Put-Outer, until the only lights left on the whole street were two tiny pinpricks in the distance, which were the eyes of the cat watching him.
Dumbledore slipped the Put-Outer back inside his cloak and set off down the street toward number four, where he sat down on the wall next to the cat.
He couldn't know that at this very moment, people meeting in secret all over the country were holding up their glasses and saying in hushed voices: "To Harry Potter — the boy who lived!"


In [12]:
from sumy.summarizers.sum_basic import SumBasicSummarizer
summarizer_5 = SumBasicSummarizer(Stemmer(language))
summarizer_5.stop_words = get_stop_words(language)
summary_5 = summarizer_5(parser.document, 5)
for sentence in summary_5:
    print(sentence)

Mr. Dursley wondered.
"Harry.
The cat was still there.
"It certainly seems so," said Dumbledore.
"Yes," said Professor McGonagall.


In [13]:
from sumy.summarizers.kl import KLSummarizer
summarizer_6 = KLSummarizer(Stemmer(language))
summarizer_6.stop_words = get_stop_words(language)
summary_6 = summarizer_6(parser.document, sentence_count)
for sentence in summary_6:
    print(sentence)

It was on the corner of the street that he noticed the first sign of something peculiar — a cat reading a map.
It grew steadily louder as they looked up and down the street for some sign of a headlight; it swelled to a roar as they both looked up at the sky — and a huge motorcycle fell out of the air and landed on the road in front of them.
He looked simply too big to be allowed, and so wild — long tangles of bushy black hair and beard hid most of his face, he had hands the size of trash can lids, and his feet in their leather boots were like baby dolphins.
"But I c-c-can't stand it — Lily an' James dead — an' poor little Harry off ter live with Muggles —" "Yes, yes, it's all very sad, but get a grip on yourself, Hagrid, or we'll be found," Professor McGonagall whispered, patting Hagrid gingerly on the arm as Dumbledore stepped over the low garden wall and walked to the front door.
He clicked it once, and twelve balls of light sped back to their street lamps so that Privet Drive glowed

In [14]:
from sumy.summarizers.reduction import ReductionSummarizer
summarizer_7 = ReductionSummarizer(Stemmer(language))
summarizer_7.stop_words = get_stop_words(language)
summary_7 = summarizer_7(parser.document, sentence_count)
for sentence in summary_7:
    print(sentence)

Mrs. Potter was Mrs. Dursley's sister, but they hadn't met for several years; in fact, Mrs. Dursley pretended she didn't have a sister, because her sister and her good-for-nothing husband were as unDursleyish as it was possible to be.
It seemed that Professor McGonagall had reached the point she was most anxious to discuss, the real reason she had been waiting on a cold, hard wall all day, for neither as a cat nor as a woman had she fixed Dumbledore with such a piercing stare as she did now.
Dumbledore took Harry in his arms and turned toward the Dursleys' house.
"But I c-c-can't stand it — Lily an' James dead — an' poor little Harry off ter live with Muggles —" "Yes, yes, it's all very sad, but get a grip on yourself, Hagrid, or we'll be found," Professor McGonagall whispered, patting Hagrid gingerly on the arm as Dumbledore stepped over the low garden wall and walked to the front door.
G'night, Professor McGonagall — Professor Dumbledore, sir."


In [15]:
from sumy.summarizers.lex_rank import LexRankSummarizer 
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.summarizers.edmundson import EdmundsonSummarizer
from sumy.summarizers.sum_basic import SumBasicSummarizer
from sumy.summarizers.kl import KLSummarizer
from sumy.summarizers.reduction import ReductionSummarizer

In [16]:
def summarize(text, summarizer, sentence_count, bonus_words=['harry', 'potter'], language='english'):
    summarizer = summarizer(Stemmer(language))
    summarizer.stop_words = get_stop_words(language)
    if isinstance(summarizer, EdmundsonSummarizer):
        summarizer.bonus_words = bonus_words
        summarizer.stigma_words = ['zdfgthdvndadv']
        summarizer.null_words = stop_words
    summary = summarizer(PlaintextParser(text, Tokenizer(language)).document, sentence_count)
    return summary

In [17]:
def print_summary(summary):
    for sentence in summary:
        print(sentence)

In [18]:
for summarizer in [LexRankSummarizer, LuhnSummarizer, LsaSummarizer, TextRankSummarizer,
                   EdmundsonSummarizer, SumBasicSummarizer, KLSummarizer, ReductionSummarizer]:
    print('----' + summarizer.__name__ + '----')
    print_summary(summarize(hp["Harry Potter and the Sorcerer's Stone"]['Chapter 1'][1],
                            summarizer,
                            sentence_count,
                            bonus_words=hp["Harry Potter and the Sorcerer's Stone"]['Chapter 1'][0].split()))
    print()

----LexRankSummarizer----
"The Potters, that's right, that's what I heard —" "— yes, their son, Harry —" Mr. Dursley stopped dead.
Twelve times he clicked the Put-Outer, until the only lights left on the whole street were two tiny pinpricks in the distance, which were the eyes of the cat watching him.
Dumbledore slipped the Put-Outer back inside his cloak and set off down the street toward number four, where he sat down on the wall next to the cat.
"But I c-c-can't stand it — Lily an' James dead — an' poor little Harry off ter live with Muggles —" "Yes, yes, it's all very sad, but get a grip on yourself, Hagrid, or we'll be found," Professor McGonagall whispered, patting Hagrid gingerly on the arm as Dumbledore stepped over the low garden wall and walked to the front door.
Dumbledore turned and walked back down the street.

----LuhnSummarizer----
It was now reading the sign that said Privet Drive — no, looking at the sign; cats couldn't read maps or signs.
He didn't see the owls swoop-

/anaconda3/lib/python3.6/site-packages/sumy/summarizers/text_rank.py:68: RuntimeWarning: invalid value encountered in true_divide
  weights /= weights.sum(axis=1)[:, numpy.newaxis]


Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much.
They were the last people you'd expect to be involved in anything strange or mysterious, because they just didn't hold with such nonsense.
Mr. Dursley was the director of a firm called Grunnings, which made drills.
He was a big, beefy man with hardly any neck, although he did have a very large mustache.
Mrs. Dursley was thin and blonde and had nearly twice the usual amount of neck, which came in very useful as she spent so much of her time craning over garden fences, spying on the neighbors.

----EdmundsonSummarizer----
The Dursleys shuddered to think what the neighbors would say if the Potters arrived in the street.
When Dudley had been put to bed, he went into the living room in time to catch the last report on the evening news: "And finally, bird-watchers everywhere have reported that the nation's owls have been behaving very unusually today.
Twelve times he cl

In [19]:
# Chapter summaries for all books
for book in hp:
    print(book)
    for chapter in hp[book]:
        print()
        print('{} - {}'.format(chapter, hp[book][chapter][0]))
        print_summary(summarize(hp[book][chapter][1],
                                EdmundsonSummarizer,
                                2,
                                bonus_words=hp[book][chapter][0].split()))
    print()
    print()

Harry Potter and the Sorcerer's Stone

Chapter 1 - THE BOY WHO LIVED
Twelve times he clicked the Put-Outer, until the only lights left on the whole street were two tiny pinpricks in the distance, which were the eyes of the cat watching him.
He couldn't know that at this very moment, people meeting in secret all over the country were holding up their glasses and saying in hushed voices: "To Harry Potter — the boy who lived!"

Chapter 2 - THE VANISHING GLASS
The sun rose on the same tidy front gardens and lit up the brass number four on the Dursleys' front door; it crept into their living room, which was almost exactly the same as it had been on the night when Mr. Dursley had seen that fateful news report about the owls.
Half an hour later, Harry, who couldn't believe his luck, was sitting in the back of the Dursleys' car with Piers and Dudley, on the way to the zoo for the first time in his life.

Chapter 3 - THE LETTERS FROM NO ONE
Yet here it was, a letter, addressed so plainly there 

Then, without warning, he leapt up and started banging his head furiously on the window, shouting, "Bad Dobby!
Dobby has heard of your greatness, sir, but of your goodness, Dobby never knew.

Chapter 3 - THE BURROW
The car revved louder and louder and suddenly, with a crunching noise, the bars were pulled clean out of the window as Fred drove straight up in the air.
As long as he wasn't intending to fly the car, the fact that the car could fly wouldn't —" "Arthur Weasley, you made sure there was a loophole when you wrote that law!"

Chapter 4 - AT FLOURISH AND BLOTTS
second-year students will require: The Standard Book of Spells, Grade 2 by Miranda Goshawk Break with a Banshee by Gilderoy Lockhart Gadding with Ghouls by Gilderoy Lockhart Holidays with Hags by Gilderoy Lockhart Travels with Trolls by Gilderoy Lockhart Voyages with Vampires by Gilderoy Lockhart Wanderings with Werewolves by Gilderoy Lockhart Year with the Yeti by Gilderoy Lockhart Fred, who had finished his own list, pee

Harry Potter was a highly unusual boy in many ways.
Arthur Weasley, Head of the Misuse of Muggle Artifacts Office at the Ministry of Magic, has won the annual Daily Prophet Grand Prize Galleon Draw.

Chapter 2 - AUNT MARGE'S BIG MISTAKE
Aunt Marge and Aunt Petunia kissed, or rather, Aunt Marge bumped her large jaw against Aunt Petunia's bony cheekbone.
yelled Uncle Vernon and Aunt Petunia together as Aunt Marge's whole body began to rise off her chair toward the ceiling.

Chapter 3 - THE KNIGHT BUS
There was another tremendous BANG, and the next moment Harry found himself flat on his bed, thrown backward by the speed of the Knight Bus.
"Anyway, they cornered Black in the middle of a street full of Muggles an' Black took out 'is wand and 'e blasted 'alf the street apart, an' a wizard got it, an' so did a dozen Muggles what got in the way.

Chapter 4 - THE LEAKY CAULDRON
Harry ate breakfast each morning in the Leaky Cauldron, where he liked watching the other guests: funny little witches

"I found this at the base of the Whomping Willow," said Snape, throwing the cloak aside, careful to keep this wand pointing directly at Lupin's chest.
Crookshanks leapt lightly off the bed and led the way out of the room, his bottlebrush tail held jauntily high.

Chapter 20 - THE DEMENTOR'S KISS
The grounds were very dark now; the only light came from the distant windows of the castle.
As the werewolf wrenched itself free of the manacle binding it, the dog seized it about the neck and pulled it backward, away from Ron and Pettigrew.

Chapter 21 - HERMIONE'S SECRET
Shocking business .
This is it, Hermione —" Hermione put her hands on Buckbeak's back and Harry gave her a leg up.

Chapter 22 - OWL POST AGAIN
I don't know whether they're used to owl post.
I'll write again soon.


Harry Potter and the Goblet of Fire

Chapter 1 - THE RIDDLE HOUSE
But over in the neighboring town of Great Hangleton, in the dark and dingy police station, Frank was stubbornly repeating, again and again, that he

As they recrossed the entrance hall with the rest of the Hogwarts students heading for the Great Hall, Harry saw Lee Jordan jumping up and down on the soles of his feet to get a better look at the back of Krum's head.
And the Goblet of Fire turned red once more; sparks showered out of it; the tongue of flame shot high into the air, and from its tip Dumbledore pulled the third piece of parchment.

Chapter 17 - THE FOUR CHAMPIONS
Harry heard the buzzing of the hundreds of students on the other side of the wall, before Professor McGonagall closed the door.
He wrenched the hangings shut around his four-poster, leaving Harry standing there by the door, staring at the dark red velvet curtains, now hiding one of the few people he had been sure would believe him.

Chapter 18 - THE WEIGHING OF THE WANDS
The prospect of going down into the Great Hall and facing the rest of the Gryffindors, all treating him like some sort of hero, was not inviting; it was that, however, or stay here and allow him

...

They did indeed burn sage and mallowsweet there on the classroom floor, and Firenze told them to look for certain shapes and symbols in the pungent fumes, but he seemed perfectly unconcerned that not one of them could see any of the signs he described, telling them that humans were hardly ever good at this, that it took centaurs years and years to become competent, and finished by telling them that it was foolish to put too much faith in such things anyway, because even centaurs sometimes read them wrongly.
He cursed and stamped on the hem of his cloak, which had started to smoke, and Marietta gave a wail and pulled the neck of her robes right up to her eyes, but not before the whole room had seen that her face was horribly disfigured by a series of close-set purple pustules that had spread across her nose and cheeks to form the word "SNEAK."

Chapter 28 - SNAPE'S WORST MEMORY
Snape remained close by, still buried in his examination questions; but this was Snape's memory, and Harry was

Harry, this is an old friend and colleague of mine, Horace Slughorn."
Harry had been sure Slughorn would be one of those wizards who could not bear to hear Voldemort's name spoken aloud, and was not disappointed: Slughorn gave a shudder and a squawk of protest, which Harry ignored.

Chapter 5 - AN EXCESS OF PHLEGM
Harry and Dumbledore approached the back door of the Burrow, which was surrounded by the familiar litter of old Wellington boots and rusty cauldrons; Harry could hear the soft clucking of sleepy chickens coming from a distant shed.
She turned to look at a large clock that was perched awkwardly on top of a pile of sheets in the washing basket at the end of the table.

Chapter 6 - DRACO'S DETOUR
Harry interrupted, with an unpleasant, hollow sensation in the pit of his stomach.
said Mr. Weasley, glaring angrily at the amulet seller.

Chapter 7 - THE SLUG CLUB
Harry spent a lot of the last week of the holidays pondering the meaning of Malfoy's behavior in Knockturn Alley.
Harry s

Exhausted but delighted with his night's work, Harry told Ron and Hermione everything that had happened during next morning's Charms lesson (having first cast the Muffliato spell upon those nearest them).
Harry was about to put his book away again when he noticed the corner of a page folded down; turning to it, he saw the Sectumsempra spell, captioned "For Enemies," that he had marked a few weeks previously.

Chapter 25 - THE SEER OVERHEARD
On one such evening, when Ginny had retired to the library, and Harry was sitting beside the window in the common room, supposedly finishing his Herbology homework but in reality reliving a particularly happy hour he had spent down by the lake with Ginny at lunchtime, Hermione dropped into the seat between him and Ron with an unpleasantly purposeful look on her face.
He had just noticed where they were standing: There on the right was the tapestry of dancing trolls, and on the left, that smoothly impenetrable stretch of stone wall that concealed — "

"— and the locket, Master Regulus's locket, Kreacher did wrong, Kreacher failed in his orders!"
"Kreacher saw him coming out of Kreacher's cupboard with his hands full of Kreacher's treasures.

Chapter 11 - THE BRIBE
Restless and irritable, Ron had developed an annoying habit of playing with the Deluminator in his pocket: This particularly infuriated Hermione, who was whiling away the wait for Kreacher by studying The Tales of Beedle the Bard and did not appreciate the way the lights kept flashing on and off.
He wanted to go to Godric's Hollow, even if Bathilda was in no fit state to talk to him; he wanted to visit the place where he and Dumbledore had both lost loved ones.

Chapter 12 - MAGIC IS MIGHT
"'Severus Snape, long-standing Potions master at Hogwarts School of Witchcraft and Wizardry, was today appointed headmaster in the most important of several staffing changes at the ancient school.
The painted image of Phineas Nigellus Black was able to flit between his portrait in Grimma

He saw the achingly familiar Hogsmeade High Street: dark shop fronts, and the outline of black mountains beyond the village, and the curve in the road ahead that led off toward Hogwarts, and light spilling from the windows of the Three Broomsticks, and with a lurch of the heart he remembered, with piercing accuracy, how he had landed here nearly a year before, supporting a desperately weak Dumbledore; all this in a second, upon landing — and then, even as he relaxed his grip upon Ron's and Hermione's arms, it happened.
The dementors had retreated, the stars were popping out again, and the footsteps of the Death Eaters were becoming louder; but before Harry in his panic could decide what to do, there was a grinding of bolts nearby, a door opened on the left-hand side of the narrow street, and a rough voice said, "Potter, in here, quick!"

Chapter 29 - THE LOST DIADEM
Multicolored hammocks were strung from the ceiling and from a balcony that ran around the dark wood-paneled and windowles

In [20]:
chapter_titles = []
for book in hp:
    chapter_titles.append(' '.join([hp[book][chapter][0] for chapter in hp[book]]))
chapter_titles = [title.split() for title in chapter_titles]
# or
chapter_titles = [title.split() for title in [' '.join([hp[book][chapter][0] for chapter in hp[book]]) for book in hp]]

In [21]:
book_text = [(' '.join([hp[book][chapter][1] for chapter in hp[book]])) for book in hp]

In [22]:
# Book summaries for all books
for book in hp:
    print(book)
    print()
    print_summary(summarize((' '.join([hp[book][chapter][1] for chapter in hp[book]])),
                            EdmundsonSummarizer,
                            5,
                            bonus_words=[title.split()
                                         for title in [' '.join([hp[book][chapter][0]
                                                                 for chapter in hp[book]])
                                                       for book in hp]]))
    print()
    print()

Harry Potter and the Sorcerer's Stone

The guard had never heard of Hogwarts and when Harry couldn't even tell him what part of the country it was in, he started to get annoyed, as though Harry was being stupid on purpose.
Harry could hear the drone of hundreds of voices from a doorway to the right — the rest of the school must already be here — but Professor McGonagall showed the first years into a small, empty chamber off the hall.
And the school bellowed: "Hogwarts, Hogwarts, Hoggy Warty Hogwarts, Teach us something please, Whether we be old and bald Or young with scabby knees, Our heads could do with filling With some interesting stuff, For now they're bare and full of air, Dead flies and bits of fluff, So teach us things worth knowing, Bring back what we've forgot, Just do your best, we'll do the rest, And learn until our brains all rot."
And then the story started to spread: Harry Potter, the famous Harry Potter, their hero of two Quidditch matches, had lost them all those points

Less than a year later a harassed-looking Fudge had appeared out of thin air in the cabinet room to inform the Prime Minister that there had been a spot of bother at the Kwidditch (or that was what it had sounded like) World Cup and that several Muggles had been "involved," but that the Prime Minister was not to worry, the fact that You-KnowWho's Mark had been seen again meant nothing; Fudge was sure it was an isolated incident, and the Muggle Liaison Office was dealing with all memory modifications as they spoke.
You can carry my words back to the others who whisper behind my back, and carry false tales of my treachery to the Dark Lord!
"But you didn't return when he came back, you didn't fly back to him at once when you felt the Dark Mark burn —" "Correct.
He was not fooled; for all Scrimgeour's talk that they had just been in the area, that Percy wanted to look up his family, this must be the real reason that they had come, so that Scrimgeour could speak to Harry alone.
Seizing the 

# Gensim

In [23]:
from gensim.summarization.summarizer import summarize
from gensim.summarization import keywords

In [24]:
print(summarize(hp["Harry Potter and the Sorcerer's Stone"]['Chapter 1'][1], ratio=0.02)) #word_count=300))

It was now reading the sign that said Privet Drive — no, looking at the sign; cats couldn't read maps or signs.
Dumbledore slipped the Put-Outer back inside his cloak and set off down the street toward number four, where he sat down on the wall next to the cat.
They're a kind of Muggle sweet I'm rather fond of." "No, thank you," said Professor McGonagall coldly, as though she didn't think this was the moment for lemon drops.
All this 'You-Know-Who' nonsense — for eleven years I have been trying to persuade people to call him by his proper name: Voldemort." Professor McGonagall flinched, but Dumbledore, who was unsticking two lemon drops, seemed not to notice.
Professor McGonagall shot a sharp look at Dumbledore and said, "The owls are nothing next to the rumors that are flying around.
It seemed that Professor McGonagall had reached the point she was most anxious to discuss, the real reason she had been waiting on a cold, hard wall all day, for neither as a cat nor as a woman had she fi

In [25]:
print (keywords(hp["Harry Potter and the Sorcerer's Stone"]['Chapter 1'][1], ratio=0.025))

dursley
dursleys
dumbledore
mcgonagall
harry
said
professor
mrs
hagrid
look
looking
looked
potters
potter
people
little
cat
cats


In [26]:
# Chapter summaries for all books
for book in hp:
    print(book)
    for chapter in hp[book]:
        print()
        print('{} - {}'.format(chapter, hp[book][chapter][0]))
        print(summarize(hp[book][chapter][1], ratio=0.01))
    print()
    print()

Harry Potter and the Sorcerer's Stone

Chapter 1 - THE BOY WHO LIVED
All this 'You-Know-Who' nonsense — for eleven years I have been trying to persuade people to call him by his proper name: Voldemort." Professor McGonagall flinched, but Dumbledore, who was unsticking two lemon drops, seemed not to notice.
Professor McGonagall shot a sharp look at Dumbledore and said, "The owls are nothing next to the rumors that are flying around.
It seemed that Professor McGonagall had reached the point she was most anxious to discuss, the real reason she had been waiting on a cold, hard wall all day, for neither as a cat nor as a woman had she fixed Dumbledore with such a piercing stare as she did now.

Chapter 2 - THE VANISHING GLASS
Ten years ago, there had been lots of pictures of what looked like a large pink beach ball wearing different-colored bonnets — but Dudley Dursley was no longer a baby, and now the photographs showed a large blond boy riding his first bicycle, on a carousel at the fair,

Uncle Vernon was large and neckless, with an enormous black mustache; Aunt Petunia was horse-faced and bony; Dudley was blond, pink, and porky.

Chapter 2 - DOBBY'S WARNING
"Harry Potter is humble and modest," said Dobby reverently, his orb-like eyes aglow.
"Dobby heard tell," he said hoarsely, "that Harry Potter met the Dark Lord for a second time, just weeks ago .

Chapter 3 - THE BURROW
I've asked you to stay about twelve times, and then Dad came home and said you'd got an official warning for using magic in front of Muggles —" "It wasn't me — and how did he know?" "He works for the Ministry," said Ron.
Fred climbed back into the car to pull with Ron, and Harry and George pushed from the bedroom side.
Then, Ron, you come bounding downstairs going, 'Mum, look who turned up in the night!' and she'll be all pleased to see Harry and no one need ever know we flew the car." 

Chapter 4 - AT FLOURISH AND BLOTTS
Harry got a shock the first time he looked in the mirror over the kitchen mante

Harry had to crane his neck to look up into the giant face above: It was ancient and monkeyish, with a long, thin beard that fell almost to the bottom of the wizard's sweeping stone robes, where two enormous gray feet stood on the smooth Chamber floor.
"You've got to help me, Tom," Harry said, raising Ginny's head again.
"I've waited a long time for this, Harry Potter," said Riddle.
Little Ginny's been writing in it for months and months, telling me all her pitiful worries and woes — how her brothers tease her, how she had to come to school with secondhand robes and books, how" — Riddle's eyes glinted — "how she didn't think famous, good, great Harry Potter would ever like her.

Chapter 18 - DOBBY'S REWARD
Mrs. Weasley let go of Harry, who hesitated for a moment, then walked over to the desk and laid upon it the Sorting Hat, the ruby-encrusted sword, and what remained of Riddle's diary.
"What interests me most," said Dumbledore gently, "is how Lord Voldemort managed to enchant Ginny, w

So at six o'clock that afternoon, Harry and Ron left Gryffindor Tower, passed the security trolls at a run, and headed down to the entrance hall.
"It came from over there!" said Malfoy, wiping his face, and staring at a spot some six feet to the left of Harry.
Harry's stomach turned over — Dumbledore would know exactly what had happened, if Malfoy said anything — 
Yes, I know it's a map," he said as Harry and Ron looked amazed.

Chapter 15 - THE QUIDDITCH FINAL
Harry and Ron both made furious moves toward Malfoy, but Hermione got there first — SMACK!
This meant (as Wood constantly reminded his team) that they needed to win the match by more than that amount to win the Cup. It also meant that the burden of winning fell largely on Harry, because capturing the Snitch was worth one hundred and fifty points.
Malfoy was still keeping close to Harry as he soared over the match, looking around for it — once Gryffindor was fifty points ahead — 
Professor McGonagall was sobbing harder even than 

"Mrs. Weasley," said Harry suddenly, unable to contain himself, "Hedwig hasn't arrived with a letter for me, has she?" 
"Ron," said Hermione, in an I-don't-think-you're-being-verysensitive sort of voice, "Harry doesn't want to play Quidditch right now.

Chapter 11 - ABOARD THE HOGWARTS EXPRESS
He, Ron, Fred, and George had just reached the first-floor landing on their way down to breakfast, when Mrs. Weasley appeared at the foot of the stairs, looking harassed.
"Look at this!" said Malfoy in ecstasy, holding up Ron's robes and showing Crabbe and Goyle, "Weasley, you weren't thinking of wearing these, were you?

Chapter 12 - THE TRIWIZARD TOURNAMENT
Harry, Ron, Hermione, and Neville jumped down from their carriage and dashed up the steps too, looking up only when they were safely inside the cavernous, torch-lit entrance hall, with its magnificent marble staircase.
Professor McGonagall, Deputy Headmistress and head of Gryffindor House, had come dashing out of the Great Hall; she skidded 

...

There was a flash of red light and Harry's body unfroze; he was able to push himself into a more dignified sitting position, hastily wipe the blood off his bruised face with the back of his hand, and raise his head to look up at Tonks, who was holding the Invisibility Cloak she had just pulled away.
Whatever Dumbledore said, Harry had had time to think over the summer, and had concluded that Snape's snide remarks to Sirius about remaining safely hidden while the rest of the Order of the Phoenix were off fighting Voldemort had probably been a powerful factor in Sirius rushing off to the Ministry the night that he had died.

Chapter 9 - THE HALF-BLOOD PRINCE
Hermione was immediately cleared to continue with Charms, Defense Against the Dark Arts, Transfiguration, Herbology, Arithmancy, Ancient Runes, and Potions, and shot off to a first-period Ancient Runes class without further ado.
"Take Charms," said Professor McGonagall, "and I shall drop Augusta a line reminding her that just because

Hermione had left her stool and was halfway toward Slughorn's desk before the rest of the class had realized it was time to move, and by the time Harry, Ron, and Ernie returned to the table, she had already tipped the contents of her phial into her cauldron and was kindling a fire underneath it.
But Harry did not answer; he was moving quickly through the crowd, past the place where Professor Flitwick was making squeaky attempts to position a few Ravenclaws, all of whom wanted to be near the front, past Professor Sprout, who was chivying the Hufflepuffs into line, until, by dodging around Ernie Macmillan, he managed to position himself right at the back of the crowd, directly behind Malfoy, who was taking advantage of the general upheaval to continue his argument with Crabbe, standing five feet away and looking mutinous.
As for the fact that Malfoy, Crabbe, and Goyle appeared to be going their different ways when they were usually inseparable, these things happened as people got older —

"So?" "My Lord, the Order of the Phoenix intends to move Harry Potter from his current place of safety on Saturday next, at nightfall." The interest around the table sharpened palpably: Some stiffened, others fidgeted, all gazing at Snape and Voldemort.
Yaxley had leaned forward to look down the long table at Voldemort and Snape.

Chapter 2 - IN MEMORIAM
The answers to these and many more questions are explored in the explosive new biography, The Life and Lies of Albus Dumbledore, by Rita Skeeter, exclusively interviewed by Betty Braithwaite, page 

Chapter 3 - THE DURSLEYS DEPARTING
They were dressed for traveling: Uncle Vernon in a fawn zip-up jacket, Aunt Petunia in a neat salmon-colored coat, and Dudley, Harry's large, blond, muscular cousin, in his leather jacket.
Aunt Petunia was looking at Uncle Vernon; Dudley was staring at Harry.

Chapter 4 - THE SEVEN POTTERS
Harry led them all back into the kitchen where, laughing and chattering, they settled on chairs, sat themselves upon A

Harry knew why she wanted to spin out their time on the riverbank; several times he saw her look up eagerly, and he was sure she had deluded herself into thinking that she heard footsteps through the heavy rain, but no red-haired figure appeared between the trees.
This scant news made Harry want to see Ginny so badly it felt like a stomachache; but it also made him think of Ron again, and of Dumbledore, and of Hogwarts itself, which he missed nearly as much as his ex-girlfriend.

Chapter 17 - BATHILDA'S SECRET
For a moment Harry considered suggesting they take refuge inside it, but before he could say anything Hermione murmured, "Let's go this way," and pulled him down the dark street leading out of the village in the opposite direction from which they had entered.
"Was there something you wanted to tell us?" Giving no sign that she had heard Hermione, Bathilda now shuffled a few steps closer to Harry.
"Yes," said Harry, looking down into the milky eyes fixed upon his own, "I think she

Before Harry or Luna could act, Professor McGonagall rose to her feet, pointed her wand at the groggy Death Eater, and said, "Imperio." 
"Minerva!" said a squeaky voice, and looking behind him, still shielding Luna from flying spells, Harry saw Professors Flitwick and Sprout sprinting up the corridor toward them in their night
"We shall meet you and your Ravenclaws in the Great Hall, Filius!" said Professor McGonagall, beckoning to Harry and Luna to follow her.

Chapter 31 - THE BATTLE OF HOGWARTS
However, as Harry skirted the walls, scanning the Gryffindor table for Ron and Hermione, Ernie Macmillan stood up at the Hufflepuff table and shouted, "And what if we want to stay and fight?" 
Harry moved up the Hall alongside the Gryffindor table, still looking for Ron and Hermione.
"I know what the diadem looks like, and I know where it is," said Harry, talking fast.
"D'you think you could just — just hold it in until we've got the diadem?" "Yeah — right — sorry —" said Ron, and he and Herm

# Book summaries

In [27]:
book = books[0][28:-4]
print(book)
print()
print(summarize((' '.join([hp[book][chapter][1] for chapter in hp[book]])), ratio=0.005))

Harry Potter and the Sorcerer's Stone

For a full minute the three of them stood and looked at the little bundle; Hagrid's shoulders shook, Professor McGonagall blinked furiously, and the twinkling light that usually shone from Dumbledore's eyes seemed to have gone out.
. He couldn't know that at this very moment, people meeting in secret all over the country were holding up their glasses and saying in hushed voices: "To Harry Potter — the boy who lived!" Nearly ten years had passed since the Dursleys had woken up to find their nephew on the front step, but Privet Drive had hardly changed at all.
Ten years ago, there had been lots of pictures of what looked like a large pink beach ball wearing different-colored bonnets — but Dudley Dursley was no longer a baby, and now the photographs showed a large blond boy riding his first bicycle, on a carousel at the fair, playing a computer game with his father, being hugged and kissed by his mother.
"I'm warning you," he had said, putting his la

In [28]:
book = books[1][28:-4]
print(book)
print()
print(summarize((' '.join([hp[book][chapter][1] for chapter in hp[book]])), ratio=0.005))

Harry Potter and the Chamber of Secrets

He missed the castle, with its secret passageways and ghosts, his classes (though perhaps not Snape, the Potions master), the mail arriving by owl, eating banquets in the Great Hall, sleeping in his four-poster bed in the tower dormitory, visiting the gamekeeper, Hagrid, in his cabin next to the Forbidden Forest in the grounds, and, especially, Quidditch, the most popular sport in the wizarding world (six tall goal posts, four flying balls, and fourteen players on broomsticks).
Neither of them had written to him all summer, even though Ron had said he was going to ask Harry to come and stay.
Countless times, Harry had been on the point of unlocking Hedwig's cage by magic and sending her to Ron and Hermione with a letter, but it wasn't worth the risk.
"Dobby heard tell," he said hoarsely, "that Harry Potter met the Dark Lord for a second time, just weeks ago .
I've asked you to stay about twelve times, and then Dad came home and said you'd got an

In [29]:
book = books[2][28:-4]
print(book)
print()
print(summarize((' '.join([hp[book][chapter][1] for chapter in hp[book]])), ratio=0.005))

Harry Potter and the Prisoner of Azkaban

"Secondly," said Uncle Vernon, acting as though he had not heard Harry's reply, "as Marge doesn't know anything about your abnormality, I don't want any — any funny stuff while she's here.
Harry spent the long sunny days exploring the shops and eating under the brightly colored umbrellas outside cafes, where his fellow diners were showing one another their purchases ("it's a lunascope, old boy — no more messing around with moon charts, see?") or else discussing the case of Sirius Black ("personally, I won't let any of the children out alone until he's back in Azkaban").
As the days slipped by, Harry started looking wherever he went for a sign of Ron or Hermione.
The assistant nearly cried when we said we wanted two." "What's all that, Hermione?" Harry asked, pointing at not one but three bulging bags in the chair next to her.
How's he going to get it with that thing around?" "That reminds me, you forgot your rat tonic," said Hermione, slapping 